In [ ]:
!pip install pydantic==2.9.2

## Validators

In validators `ValueError` should be used - it is translated to `ValidationError` by `BaseModel`, https://docs.pydantic.dev/latest/errors/errors/#custom-errors

In [4]:
from pydantic import BaseModel, field_validator, ValidationError

class UserModel(BaseModel):
    name: str
    role: str

    @field_validator('role')
    def validate_role(cls, value):
        allowed_roles = {'admin', 'user', 'guest'}
        if value not in allowed_roles:
            raise ValueError(f'Role must be one of {allowed_roles}')
        return value

# Example usage
try:
    user = UserModel(name='John Doe', role='manager')
except ValidationError as e:
    print(e)  # Output: Role must be one of {'admin', 'user', 'guest'}

1 validation error for UserModel
role
  Value error, Role must be one of {'guest', 'admin', 'user'} [type=value_error, input_value='manager', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error


In [13]:
from pydantic import BaseModel, model_validator, ValidationError

class UserModel(BaseModel):
    name: str
    age: int
    role: str

    @model_validator(mode='after')
    def validate_model(cls, values):
        if values.age < 18 and values.role == 'admin':
            raise ValueError('Admin role cannot be assigned to users under 18')
        return values

# Example usage
try:
    user = UserModel(name='John Doe', age=17, role='admin')
except ValidationError as e:
    print(e)  # Output: Admin role cannot be assigned to users under 18


# mode='before': Use this if you need to preprocess or validate data before individual field validation.
# mode='after': Use this for checks that depend on the final state of the model, after all fields have been validated.

1 validation error for UserModel
  Value error, Admin role cannot be assigned to users under 18 [type=value_error, input_value={'name': 'John Doe', 'age': 17, 'role': 'admin'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error


## Field

In [ ]:
from pydantic import BaseModel, Field

In [9]:
class UserModel(BaseModel):
    name: str = Field(coerce_numbers_to_str=True)

UserModel(name=1)

UserModel(name='1')

In [ ]:
class UserModel(BaseModel):
    name_of_user: str = Field(alias="nameOfUser")

UserModel(**{"nameOfUser": "Bob"})

UserModel(name_of_user='Bob')

In [52]:
class UserModel(BaseModel):
    one: str
    two: str = Field(alias="one")

user= UserModel(one="one")
user.two = "two"
user

UserModel(one='one', two='two')

In [21]:
from datetime import datetime

class UserModel(BaseModel):
    created_at: datetime = Field(default_factory=datetime.now)

UserModel()

UserModel(created_at=datetime.datetime(2025, 3, 1, 23, 31, 58, 966309))

In [47]:
class UserModel(BaseModel):
    name: str = "unknown"
    name_of_user: str = Field(alias="name", deprecated=True)

UserModel(name="Bob").name_of_user

/tmp/ipykernel_21259/2519091863.py:5: DeprecationWarning: deprecated
  UserModel(name="Bob").name_of_user


'Bob'

In [ ]:
from uuid import UUID

class MyModel(BaseModel):
    guid: UUID = Field(strict=True)

MyModel(guid=UUID('12345678-1234-1234-1234-123456789012'))  # ok
MyModel(guid='12345678-1234-1234-1234-123456789012')  # ValidationError

In [ ]:
class UserModel(BaseModel):
    code: str = Field(pattern=r"^[A-Z]{3}$")

UserModel(code="AAA")  # ok
UserModel(code="AB")  # ValidationError